create new files with for each image setting the roi associated significant

In [2]:

import numpy as np
import optuna as optuna
import pandas as pd
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
import extract_correct_csv
from tqdm import tqdm
scaler = StandardScaler()
import re

In [3]:
valid_subject = extract_correct_csv.extract_only_valid_subject()
if 49 in valid_subject:
    valid_subject.remove(49)

In [4]:
mouth = ['M078C','F080C','3&4','7&8']
eyes = ['F053C','M024C',"5&6",'1&2']
for sub in valid_subject:
    string_sub = extract_correct_csv.read_correct_subject_csv(sub)
    lookatme= pd.read_csv('data/newLookAtMe/newLookAtMe'+string_sub+'.csv')
    lookatme['ROI_change'] =np.zeros(len(lookatme))
    list_roi = []
    for row in lookatme.iterrows():
        picname = row[1][12]
        other = True
        for el in mouth:
            if el in picname:
                other=False
                list_roi.append('mouth_nose')

        for el in eyes:
            if el in picname:
                other=False
                list_roi.append('eyes')

        if other:
            list_roi.append('other')
    lookatme['ROI'] = [x for x in list_roi]
    lookatme= lookatme.drop(columns=lookatme.columns[0])
    lookatme.to_csv('data/newLookAtMeROI/LookAtMe'+string_sub+'.csv',index=False)

In [8]:
lookatme

,Unnamed: 0,expName,subject,pairs,cs1,cs2,block,trial,condition,rating,rt,shock,picName,timestamp,morphing level,gender,ROI
0,0,LookAtMe,1,1,1,1,1,1,41,1.0,1.412052,False,stim\8 F080C + F101C.jpg,2018-12-18 15:00:50,1,1,mouth_nose
1,1,LookAtMe,1,1,1,1,1,2,46,1.0,1.036975,False,stim\7 F080C.jpg,2018-12-18 15:00:59,6,1,mouth_nose
2,2,LookAtMe,1,1,1,1,1,3,16,2.0,1.146517,False,stim\1 M024C.jpg,2018-12-18 15:01:07,6,0,eyes
3,3,LookAtMe,1,1,1,1,1,4,11,3.0,1.294659,False,stim\2 M024C + M010C.jpg,2018-12-18 15:01:17,1,0,eyes
4,4,LookAtMe,1,1,1,1,1,5,41,NaN,-1.000000,False,stim\8 F080C + F101C.jpg,2018-12-18 15:01:25,1,1,mouth_nose
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,155,LookAtMe,1,1,1,1,2,156,44,2.0,0.430953,False,stim\morph\7&8 40.jpg,2018-12-18 15:27:39,4,1,other
156,156,LookAtMe,1,1,1,1,2,157,15,2.0,0.326550,False,stim\morph\1&2 20.jpg,2018-12-18 15:27:48,5,0,other
157,157,LookAtMe,1,1,1,1,2,158,41,1.0,0.351921,False,stim\8 F080C + F101C.jpg,2018-12-18 15:27:58,1,1,mouth_nose
158,158,LookAtMe,1,1,1,1,2,159,11,3.0,1.515047,False,stim\2 M024C + M010C.jpg,2018-12-18 15:28:07,1,0,eyes


## logistic regression with sklearn

In [3]:
columns = ['subject', 'feature','fold', 'train accuracy', 'test accuracy']
results = pd.DataFrame(columns = columns)
index = 1
for x in valid_subject:



    types_ = ['hr', 'eda', 'pupil']
    for type_ in types_:
        X = pd.read_csv('data/features_4_2/'+type_+'/45.csv')
        X = X[48:]
        X = scaler.fit_transform(X)
        # creating train and test
        X = pd.DataFrame(X)
        X = X.reset_index().drop(columns=('index'))



        sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=123)

        for i, (train_index, test_index) in enumerate(sss.split(X, y)):
            y = pd.DataFrame(y)
            y = y.reset_index().drop(columns=('index'))

            N_train = len(train_index)
            X_train = X.iloc[train_index, :]
            y_train =y.iloc[train_index, :]

            X_test = X.iloc[test_index, :]
            y_test =y.iloc[test_index, :]

            clf = LogisticRegression(random_state=123,max_iter=10000)
            clf.fit(X_train, np.squeeze(np.array(y_train)))

            train_ = clf.score(X_train, y_train)
            test_ = clf.score(X_test, y_test)
            dict_ = {'subject': x, 'feature': type_,'fold':int(i), 'train accuracy': train_, 'test accuracy': test_}
            results = pd.concat([results, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)
            index = index + 1

NameError: name 'y' is not defined

## accuracy logistic classifier sklearn monophysio

In [ ]:
reg_mono = results
reg_mono

In [ ]:
columns = ['subject', 'fold']

In [ ]:
mean_regr=reg_mono.drop(columns=columns).groupby(by='feature', as_index=False).mean()
var_regr=reg_mono.drop(columns=columns).groupby(by='feature', as_index=False).var()

## mean accuracy over all subjs logistic classifier sklearn monophysio

In [ ]:
mean_regr

## var over all subjs logistic classifier sklearn monophysio


In [ ]:
var_regr

# gaze -> label classifier with roi

In [20]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
subjects = extract_correct_csv.extract_only_valid_subject()
subjects.remove(21)
subjects.remove(22)

def custom_parse_data(X):
    res_x = []
    for i in range(len(X)):
        string = X[i]
        new_string = re.sub(r'\s+', ',', string)
        string_list = list(new_string)
        string_list[1] = ''
        new_string = ''.join(string_list)
        res_x.append(np.fromstring(new_string.strip('[]'), dtype=float, sep=','))

    return np.array(res_x)

df_ = pd.read_csv('data/gaze/joined_fixation.csv')
# remove index column
df_ = df_.drop(columns=df_.columns[0])
# replace NaN with 'other'
df_['ROI'] = df_['ROI'].replace(np.NaN, 'other')
# parse fixation feature column to convert this in value
feat = custom_parse_data(df_['Fixation feature'])
df_['Fixation feature'] = [x for x in feat]
# mean features with same ROI
result = df_.groupby(['Subject', 'Trial', 'ROI'], as_index=False)['Fixation feature'].mean()
# insert an array of zeros if one subject in one trial has no fixations on that ROI
result_modified = result.copy(deep=True)
for sub in result['Subject'].unique():
    df = result[result['Subject'] == sub]
    for trial in df['Trial'].unique():
        if 'mouth_nose' not in list(df[df['Trial'] == trial]['ROI']):
            result_modified = result_modified.append(
                {'Subject': sub, 'Trial': trial, 'ROI': 'mouth_nose', 'Fixation feature': [0] * 13}, ignore_index=True)
        if 'eye' not in list(df[df['Trial'] == trial]['ROI']):
            result_modified = result_modified.append(
                {'Subject': sub, 'Trial': trial, 'ROI': 'eye', 'Fixation feature': [0] * 13}, ignore_index=True)
        if 'other' not in list(df[df['Trial'] == trial]['ROI']):
            result_modified = result_modified.append(
                {'Subject': sub, 'Trial': trial, 'ROI': 'other', 'Fixation feature': [0] * 13}, ignore_index=True)

result_modified = result_modified.sort_values(by = ['Subject', 'Trial','ROI']).reset_index(drop=True)


/tmp/ipykernel_57663/1882881891.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_modified = result_modified.append(
/tmp/ipykernel_57663/1882881891.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_modified = result_modified.append(
/tmp/ipykernel_57663/1882881891.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_modified = result_modified.append(
/tmp/ipykernel_57663/1882881891.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_modified = result_modified.append(
/tmp/ipykernel_57663/1882881891.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future 

In [22]:
result_modified

,Subject,Trial,ROI,Fixation feature
0,1,1,eye,"[0.10957644862499998, -0.05454303021875, 0.193..."
1,1,1,mouth_nose,"[0.19989683437142855, -0.05938805191428571, 0...."
2,1,1,other,"[0.19819834038125, -0.07207240349374999, 0.242..."
3,1,2,eye,"[0.20223466344285715, -0.040219880971428576, 0..."
4,1,2,mouth_nose,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...
22489,55,159,mouth_nose,"[0.0711068112, -0.05118723756, 0.16613707786, ..."
22490,55,159,other,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
22491,55,160,eye,"[0.19023594441875002, -0.12665187247499998, 0...."
22492,55,160,mouth_nose,"[0.0560755235, -0.0006939780999999978, 0.36795..."


In [16]:
columns_multi = ['subject', 'fold', 'train accuracy', 'test accuracy']
reg_multi = pd.DataFrame(columns = columns_multi)
index = 1

for sub in subjects:

    string_sub = extract_correct_csv.read_correct_subject_csv(sub)

    df_ = pd.read_csv('data/LookAtMe_old/LookAtMe_0'+string_sub+'.csv', sep='\t')
    y = np.array(list([int (d > 2) for d in df_['rating']]))
    y = y[48:]

    # gaze data
    df_sub = result_modified[result_modified['Subject'] == sub]
    # normalize feature and convert from [13][13][13] into [39]
    X1gaze = df_sub[df_sub['ROI'] == 'eye']
    X1_normgaze = pd.DataFrame(scaler.fit_transform(list(X1gaze['Fixation feature'])))
    X2gaze = df_sub[df_sub['ROI'] == 'mouth_nose']
    X2_normgaze = pd.DataFrame(scaler.fit_transform(list(X2gaze['Fixation feature'])))
    X3gaze = df_sub[df_sub['ROI'] == 'other']
    X3_normgaze = pd.DataFrame(scaler.fit_transform(list(X3gaze['Fixation feature'])))
    X_norm = pd.concat([X1gaze, X2gaze, X3gaze], axis=1)

    lookatme= pd.read_csv('data/newLookAtMe/newLookAtMe'+string_sub+'.csv')
    #TODO join lookatme and X_norm
    #significan_ROI = ,
    X_norm = pd.concat([X_norm,significant_ROI])
    # remove first 48 learning trials
    #X_norm = X_norm[48:]
    X_norm = pd.DataFrame(X_norm)
    X_norm = X_norm.reset_index().drop(columns=('index'))
    gaze = X_norm


    X = gaze
    #X = pd.concat([X,gaze],axis=1)
    X = X[48:]
    X = pd.DataFrame(X)
    X = X.reset_index().drop(columns=('index'))


    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=123)

    for i, (train_index, test_index) in enumerate(sss.split(X, y)):
        y = pd.DataFrame(y)
        y = y.reset_index().drop(columns=('index'))

        N_train = len(train_index)
        X_train = X.iloc[train_index, :]
        y_train =y.iloc[train_index, :]

        X_test = X.iloc[test_index, :]
        y_test =y.iloc[test_index, :]

        classifier = LogisticRegression(max_iter=10000,random_state=123)
        classifier.fit(X_train, np.squeeze(np.array(y_train)))

        train_ = classifier.score(X_train, y_train)
        test_ = classifier.score(X_test, y_test)
        dict_ = {'subject': sub, 'fold':int(i), 'train accuracy': train_, 'test accuracy': test_}
        reg_multi = pd.concat([reg_multi, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)
        index = index + 1

NameError: name 'result_modified' is not defined

In [28]:
X1norm

,Subject,Trial,ROI,Fixation feature
0,1,1,eye,"[0.10957644862499998, -0.05454303021875, 0.193..."
3,1,2,eye,"[0.20223466344285715, -0.040219880971428576, 0..."
6,1,3,eye,"[1.7977614672181816, -3.444527689963636, 12.30..."
9,1,4,eye,"[0.07814237955, -0.05707772527500002, 0.260199..."
12,1,5,eye,"[0.18118808552857144, -0.07466363480000002, 0...."
...,...,...,...,...
465,1,156,eye,"[0.201129310825, -0.0620863934825, 0.226575993..."
468,1,157,eye,"[0.08835547614210526, -0.06808866378947369, 0...."
471,1,158,eye,"[0.16394864024, -0.06737593790799999, 0.237059..."
474,1,159,eye,"[0.17625440033333337, -0.07695262154083332, 0...."


In [27]:
X1gaze

,Subject,Trial,ROI,Fixation feature
0,1,1,eye,"[0.10957644862499998, -0.05454303021875, 0.193..."
3,1,2,eye,"[0.20223466344285715, -0.040219880971428576, 0..."
6,1,3,eye,"[1.7977614672181816, -3.444527689963636, 12.30..."
9,1,4,eye,"[0.07814237955, -0.05707772527500002, 0.260199..."
12,1,5,eye,"[0.18118808552857144, -0.07466363480000002, 0...."
...,...,...,...,...
465,1,156,eye,"[0.201129310825, -0.0620863934825, 0.226575993..."
468,1,157,eye,"[0.08835547614210526, -0.06808866378947369, 0...."
471,1,158,eye,"[0.16394864024, -0.06737593790799999, 0.237059..."
474,1,159,eye,"[0.17625440033333337, -0.07695262154083332, 0...."


In [ ]:
X

In [25]:

grouped_df = reg_multi.groupby(by=['subject'], as_index=False).mean()
grouped_df

## mean accuracy over all subjs gaze -> label

In [7]:
grouped_df.drop(columns=['subject']).mean()

train accuracy    0.727662
test accuracy     0.569876
dtype: float64

## var accuracy over all subjs gaze -> label

In [8]:
grouped_df.drop(columns=['subject']).var()

train accuracy    0.002041
test accuracy     0.005991
dtype: float64

# multi physio and gaze classifier

In [9]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [10]:
subjects = extract_correct_csv.extract_only_valid_subject()
subjects.remove(21)
subjects.remove(22)

def custom_parse_data(X):
    res_x = []
    for i in range(len(X)):
        string = X[i]
        new_string = re.sub(r'\s+', ',', string)
        string_list = list(new_string)
        string_list[1] = ''
        new_string = ''.join(string_list)
        res_x.append(np.fromstring(new_string.strip('[]'), dtype=float, sep=','))

    return np.array(res_x)

df_ = pd.read_csv('data/gaze/joined_fixation.csv')
# remove index column
df_ = df_.drop(columns=df_.columns[0])
# replace NaN with 'other'
df_['ROI'] = df_['ROI'].replace(np.NaN, 'other')
# parse fixation feature column to convert this in value
feat = custom_parse_data(df_['Fixation feature'])
df_['Fixation feature'] = [x for x in feat]
# mean features with same ROI
result = df_.groupby(['Subject', 'Trial', 'ROI'], as_index=False)['Fixation feature'].mean()
# insert an array of zeros if one subject in one trial has no fixations on that ROI
result_modified = result.copy(deep=True)
for sub in result['Subject'].unique():
    df = result[result['Subject'] == sub]
    for trial in df['Trial'].unique():
        if 'mouth_nose' not in list(df[df['Trial'] == trial]['ROI']):
            result_modified = result_modified.append(
                {'Subject': sub, 'Trial': trial, 'ROI': 'mouth_nose', 'Fixation feature': [0] * 13}, ignore_index=True)
        if 'eye' not in list(df[df['Trial'] == trial]['ROI']):
            result_modified = result_modified.append(
                {'Subject': sub, 'Trial': trial, 'ROI': 'eye', 'Fixation feature': [0] * 13}, ignore_index=True)
        if 'other' not in list(df[df['Trial'] == trial]['ROI']):
            result_modified = result_modified.append(
                {'Subject': sub, 'Trial': trial, 'ROI': 'other', 'Fixation feature': [0] * 13}, ignore_index=True)


columns_multi = ['subject', 'fold', 'train accuracy', 'test accuracy']
reg_multi = pd.DataFrame(columns = columns_multi)
index = 1

for sub in subjects:

    string_sub = extract_correct_csv.read_correct_subject_csv(sub)

    df_ = pd.read_csv('data/LookAtMe_old/LookAtMe_0'+string_sub+'.csv', sep='\t')
    y = np.array(list([int (d > 2) for d in df_['rating']]))
    y = y[48:]


    X1 = pd.read_csv('data/features_4_2/hr/'+str(sub)+'.csv')
    X1 = pd.DataFrame(scaler.fit_transform(X1))
    X2 = pd.read_csv('data/features_4_2/eda/'+str(sub)+'.csv')
    X2 = pd.DataFrame(scaler.fit_transform(X2))
    X3 = pd.read_csv('data/features_4_2/pupil/'+str(sub)+'.csv')
    X3 = pd.DataFrame(scaler.fit_transform(X3))

    # gaze data
    df_sub = result_modified[result_modified['Subject'] == sub]
    # normalize feature and convert from [13][13][13] into [39]
    X1gaze = df_sub[df_sub['ROI'] == 'eye']
    X1_normgaze = pd.DataFrame(scaler.fit_transform(list(X1gaze['Fixation feature'])))
    X2gaze = df_sub[df_sub['ROI'] == 'mouth_nose']
    X2_normgaze = pd.DataFrame(scaler.fit_transform(list(X2gaze['Fixation feature'])))
    X3gaze = df_sub[df_sub['ROI'] == 'other']
    X3_normgaze = pd.DataFrame(scaler.fit_transform(list(X3gaze['Fixation feature'])))
    X_norm = pd.concat([X1_normgaze, X2_normgaze, X3_normgaze], axis=1)
    # remove first 48 learning trials
    #X_norm = X_norm[48:]
    X_norm = pd.DataFrame(X_norm)
    X_norm = X_norm.reset_index().drop(columns=('index'))
    gaze = X_norm


    if X1.shape[0] == 159:
        gaze=gaze[:-1]
        y =y[:-1]
    print(X1.shape,X2.shape,X3.shape,gaze.shape)

    X = pd.concat([X1, X2, X3, gaze], axis=1)
    #X = pd.concat([X,gaze],axis=1)
    X = X[48:]
    X = pd.DataFrame(X)
    X = X.reset_index().drop(columns=('index'))


    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=123)

    for i, (train_index, test_index) in enumerate(sss.split(X, y)):
        y = pd.DataFrame(y)
        y = y.reset_index().drop(columns=('index'))

        N_train = len(train_index)
        X_train = X.iloc[train_index, :]
        y_train =y.iloc[train_index, :]

        X_test = X.iloc[test_index, :]
        y_test =y.iloc[test_index, :]

        classifier = LogisticRegression(max_iter=10000,random_state=123)
        classifier.fit(X_train, np.squeeze(np.array(y_train)))

        train_ = classifier.score(X_train, y_train)
        test_ = classifier.score(X_test, y_test)
        dict_ = {'subject': sub, 'fold':int(i), 'train accuracy': train_, 'test accuracy': test_}
        reg_multi = pd.concat([reg_multi, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)
        index = index + 1


(160, 24) (160, 24) (160, 12) (160, 39)
(160, 24) (160, 24) (160, 28) (160, 39)
(160, 24) (160, 24) (160, 28) (160, 39)
(160, 24) (160, 24) (160, 12) (160, 39)
(160, 24) (160, 24) (160, 12) (160, 39)
(160, 24) (160, 24) (160, 12) (160, 39)
(159, 134) (159, 134) (159, 28) (159, 39)
(159, 134) (159, 134) (159, 28) (159, 39)
(159, 134) (159, 134) (159, 28) (159, 39)
(159, 134) (159, 134) (159, 28) (159, 39)
(159, 134) (159, 134) (159, 28) (159, 39)
(159, 134) (159, 134) (159, 28) (159, 39)
(159, 134) (159, 134) (159, 28) (159, 39)
(159, 134) (159, 134) (159, 28) (159, 39)
(159, 134) (159, 134) (159, 28) (159, 39)
(159, 134) (159, 134) (159, 28) (159, 39)
(159, 134) (159, 134) (159, 28) (159, 39)
(160, 24) (160, 24) (160, 28) (160, 39)
(160, 24) (160, 24) (160, 28) (160, 39)
(160, 24) (160, 24) (160, 28) (160, 39)
(160, 24) (160, 24) (160, 28) (160, 39)
(160, 24) (160, 24) (160, 28) (160, 39)
(160, 24) (160, 24) (160, 28) (160, 39)
(160, 24) (160, 24) (160, 28) (160, 39)
(160, 24) (160, 24

## accuracy logistic classifier sklearn mean by subj

In [12]:
lds_df = pd.read_csv('data/lds_subjects.csv')
sias_df = pd.read_csv('data/sias_score.csv')
list_sub_sias = sias_df.sort_values(by='sias_score', ascending=True)[:10]['subject'].unique()
list_sub_lds = lds_df.sort_values(by='lds', ascending=False)[:10]['subject'].unique()

list_sub_sias_more = sias_df.sort_values(by='sias_score', ascending=False)[:10]['subject'].unique()
list_sub_lds_more = lds_df.sort_values(by='lds', ascending=True)[:10]['subject'].unique()

In [13]:
grouped_df = reg_multi.groupby(by=['subject'], as_index=False).mean()
grouped_df_less_lds = grouped_df[grouped_df.subject.isin(list_sub_lds)]
grouped_df_less_sias = grouped_df[grouped_df.subject.isin(list_sub_sias)]

grouped_df_more_lds = grouped_df[grouped_df.subject.isin(list_sub_lds_more)]
grouped_df_more_sias = grouped_df[grouped_df.subject.isin(list_sub_sias_more)]

### Low fear generalization
- lds

In [14]:
grouped_df_less_lds

,subject,train accuracy,test accuracy
4,6,0.910112,0.782609
12,27,0.943182,0.666667
13,28,0.946970,0.724638
14,31,0.946970,0.782609
27,54,0.921348,0.594203


- sias

In [15]:
grouped_df_less_sias

,subject,train accuracy,test accuracy
1,2,0.880150,0.637681
2,4,0.816479,0.536232
4,6,0.910112,0.782609
11,26,0.924242,0.666667
25,51,0.835206,0.536232


### High fear generalization
- lds

In [18]:
grouped_df_more_lds

,subject,train accuracy,test accuracy
0,1,0.838951,0.550725
2,4,0.816479,0.536232
6,13,0.905303,0.521739
9,17,0.867424,0.608696
10,23,0.867424,0.608696
15,32,0.878788,0.507246
19,44,0.880150,0.623188
21,46,0.838951,0.623188


- sias

In [19]:
grouped_df_more_sias

,subject,train accuracy,test accuracy
3,5,0.842697,0.565217
5,7,0.850187,0.362319
8,16,0.916667,0.521739
10,23,0.867424,0.608696
12,27,0.943182,0.666667
18,43,0.936330,0.579710
21,46,0.838951,0.623188


## mean accuracy over all subjs logistic classifier sklearn

In [20]:
grouped_df.drop(columns=['subject']).mean()

train accuracy    0.887799
test accuracy     0.599379
dtype: float64

In [21]:
# mean low fear gen lds
grouped_df_less_lds.drop(columns=['subject']).mean()

train accuracy    0.933716
test accuracy     0.710145
dtype: float64

In [22]:
# mean low fear gen sias
grouped_df_less_sias.drop(columns=['subject']).mean()

train accuracy    0.873238
test accuracy     0.631884
dtype: float64

In [23]:
# high fear gen lds
grouped_df_more_lds.drop(columns=['subject']).mean()

train accuracy    0.861684
test accuracy     0.572464
dtype: float64

In [24]:
# high fear gen sias
grouped_df_more_sias.drop(columns=['subject']).mean()

train accuracy    0.885063
test accuracy     0.561077
dtype: float64

## var over all subjs logistic classifier sklearn

In [ ]:
grouped_df.drop(columns=['subject']).var()

In [ ]:
print("test accuracy all physio\nmean: ",np.mean(np.array(mean_test)))
print('var: ',np.var(np.array(mean_test)))

# log regr gaze with roi associated

# Multi physio classifier

In [ ]:
subjects = extract_correct_csv.extract_only_valid_subject()
subjects

In [ ]:

columns_multi = ['subject','fold', 'train accuracy', 'test accuracy']
reg_multi = pd.DataFrame(columns = columns_multi)
index = 1

for sub in subjects:

    string_sub = extract_correct_csv.read_correct_subject_csv(sub)

    df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
    y = np.array(list([int (d > 2) for d in df_['rating']]))
    y = y[48:]


    X1 = pd.read_csv('data/features_4_2/hr/'+str(sub)+'.csv')
    X1 = pd.DataFrame(scaler.fit_transform(X1))
    X2 = pd.read_csv('data/features_4_2/eda/'+str(sub)+'.csv')
    X2 = pd.DataFrame(scaler.fit_transform(X2))
    X3 = pd.read_csv('data/features_4_2/pupil/'+str(sub)+'.csv')
    X3 = pd.DataFrame(scaler.fit_transform(X3))
    X = pd.concat([X1, X2, X3], axis=1)
    X = X[48:]

    X = pd.DataFrame(X)
    X = X.reset_index().drop(columns=('index'))

    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=123)

    for i, (train_index, test_index) in enumerate(sss.split(X, y)):
        y = pd.DataFrame(y)
        y = y.reset_index().drop(columns=('index'))

        N_train = len(train_index)
        X_train = X.iloc[train_index, :]
        y_train =y.iloc[train_index, :]

        X_test = X.iloc[test_index, :]
        y_test =y.iloc[test_index, :]

        classifier = LogisticRegression(max_iter=10000,random_state=123)
        clf.fit(X_train, np.squeeze(np.array(y_train)))

        train_ = clf.score(X_train, y_train)
        test_ = clf.score(X_test, y_test)
        dict_ = {'subject': sub, 'fold':int(i), 'train accuracy': train_, 'test accuracy': test_}
        reg_multi = pd.concat([reg_multi, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)
        index = index + 1

## accuracy logistic classifier sklearn

In [ ]:
reg_multi

In [ ]:
columns = ['subject', 'fold']

## accuracy logistic classifier sklearn mean by subj

In [ ]:
grouped_df = reg_multi.groupby(by=['subject'], as_index=False).mean()
grouped_df

## mean accuracy over all subjs logistic classifier sklearn

In [ ]:
grouped_df.drop(columns=['subject']).mean()

## var over all subjs logistic classifier sklearn

In [ ]:
grouped_df.drop(columns=['subject']).var()

In [ ]:
print("test accuracy all physio\nmean: ",np.mean(np.array(mean_test)))
print('var: ',np.var(np.array(mean_test)))

# OU ROI features

In [ ]:
def custom_parse_data(X):
  res_x =[]
  for i in range(len(X)):
    string = X[i]
    new_string = re.sub(r'\s+', ',', string)
    string_list = list(new_string)
    string_list[1] = ''
    new_string = ''.join(string_list)
    res_x.append(np.fromstring(new_string.strip('[]'), dtype=float, sep=','))

  return np.array(res_x)

In [ ]:
df_ = pd.read_csv('data/gaze/joined_fixation.csv')
# remove index column
df_ = df_.drop(columns=df_.columns[0])
# replace NaN with 'other'
df_['ROI'] = df_['ROI'].replace(np.NaN, 'other')
# parse fixation feature column to convert this in value
feat = custom_parse_data(df_['Fixation feature'])
df_['Fixation feature'] = [x for x in feat]

In [ ]:
# mean features with same ROI
result = df_.groupby(['Subject', 'Trial', 'ROI'],as_index=False)['Fixation feature'].mean()

In [ ]:
# insert an array of zeros if one subject in one trial has no fixations on that ROI

result_modified = result.copy(deep=True)

for sub in result['Subject'].unique():
    df = result[result['Subject'] ==sub]

    for trial in df['Trial'].unique():

        if 'mouth_nose' not in list(df[df['Trial']==trial]['ROI']):
            result_modified = result_modified.append({'Subject':sub,'Trial':trial,'ROI':'mouth_nose', 'Fixation feature': [0]*13},ignore_index=True)
        if 'eye' not in list(df[df['Trial']==trial]['ROI']):
            result_modified = result_modified.append({'Subject':sub,'Trial':trial,'ROI':'eye', 'Fixation feature': [0]*13},ignore_index=True)
        if 'other' not in list(df[df['Trial']==trial]['ROI']):
            result_modified = result_modified.append({'Subject':sub,'Trial':trial,'ROI':'other', 'Fixation feature': [0]*13},ignore_index=True)

# sklearn logistic regression OU roi features

In [ ]:
index = 1

In [ ]:
reg_multi = pd.DataFrame(columns=['subject','fold','train accuracy','test accuracy'])


for sub in valid_subject:

    string_sub = extract_correct_csv.read_correct_subject_csv(sub)

    # data of one subject
    df_sub = result_modified[result_modified['Subject']==sub]

    # convert rating into 0/1
    df_look = pd.read_csv('./data/LookAtMe_old/LookAtMe_0'+string_sub+'.csv', sep='\t')
    y = np.array(list([int (d > 2) for d in df_look['rating']]))
    y = y[48:]
    # normalize feature and convert from [13][13][13] into [39]
    X1 = df_sub[df_sub['ROI']=='eye']
    X1_norm = pd.DataFrame(scaler.fit_transform(list(X1['Fixation feature'])))
    X2 = df_sub[df_sub['ROI']=='mouth_nose']
    X2_norm = pd.DataFrame(scaler.fit_transform(list(X2['Fixation feature'])))
    X3 = df_sub[df_sub['ROI'] =='other']
    X3_norm = pd.DataFrame(scaler.fit_transform(list(X3['Fixation feature'])))
    X_norm = pd.concat([X1_norm, X2_norm, X3_norm], axis=1)
    # remove first 48 learning trials
    X_norm = X_norm[48:]
    X_norm = pd.DataFrame(X_norm)
    X_norm = X_norm.reset_index().drop(columns=('index'))

    #kcv
    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=123)

    print(sub,X_norm.shape,y.shape)
    for i, (train_index, test_index) in enumerate(sss.split(X_norm, y)):
        y = pd.DataFrame(y)
        y = y.reset_index().drop(columns=('index'))

        N_train = len(train_index)
        X_train = X_norm.iloc[train_index, :]
        y_train =y.iloc[train_index, :]

        X_test = X_norm.iloc[test_index, :]
        y_test =y.iloc[test_index, :]

        clf = LogisticRegression(max_iter=10000,random_state=123)
        clf.fit(X_train, np.squeeze(np.array(y_train)))

        train_ = clf.score(X_train, y_train)
        test_ = clf.score(X_test, y_test)
        dict_ = {'subject': sub, 'fold':int(i), 'train accuracy': train_, 'test accuracy': test_}
        reg_multi = pd.concat([reg_multi, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)
        index = index + 1

In [ ]:
acc = reg_multi.groupby(['subject']).mean()
acc

In [36]:
acc[acc['test accuracy']>0.6]

,train accuracy,test accuracy
subject,,
2,0.715356,0.623188
6,0.805243,0.782609
17,0.734082,0.695652
21,0.737828,0.652174
27,0.737828,0.666667
31,0.797753,0.652174
45,0.786517,0.652174
51,0.677903,0.608696
54,0.722846,0.608696


# MLP OU roi features

In [38]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
valid_subject = extract_correct_csv.extract_only_valid_subject()

In [39]:
# MLP mono signals
columns_mono = ['Subject', 'Train', 'Test']
MLP_df_gaze = pd.DataFrame(columns=columns_mono)

for sub in valid_subject:

    string_sub = extract_correct_csv.read_correct_subject_csv(sub)

    # data of one subject
    df_sub = result_modified[result_modified['Subject']==sub]

    # convert rating into 0/1
    df_look = pd.read_csv('./data/LookAtMe_old/LookAtMe_0'+string_sub+'.csv', sep='\t')
    y = np.array(list([int (d > 2) for d in df_look['rating']]))
    y = y[48:]
    # normalize feature and convert from [13][13][13] into [39]
    X1 = df_sub[df_sub['ROI']=='eye']
    X1_norm = pd.DataFrame(scaler.fit_transform(list(X1['Fixation feature'])))
    X2 = df_sub[df_sub['ROI']=='mouth_nose']
    X2_norm = pd.DataFrame(scaler.fit_transform(list(X2['Fixation feature'])))
    X3 = df_sub[df_sub['ROI'] =='other']
    X3_norm = pd.DataFrame(scaler.fit_transform(list(X3['Fixation feature'])))
    X_norm = pd.concat([X1_norm, X2_norm, X3_norm], axis=1)
    # remove first 48 learning trials
    X_norm = X_norm[48:]
    X_norm = pd.DataFrame(X_norm)
    X_norm = X_norm.reset_index().drop(columns=('index'))


    X_train, X_test,y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=123, stratify=y)
    mlp_clf = MLPClassifier(hidden_layer_sizes=(30, 20), max_iter=3000, learning_rate='adaptive', random_state=123).fit(X_train, y_train)
    row_ = {'Subject': sub,
            'Train': mlp_clf.score(X_train, y_train),
            'Test':mlp_clf.score(X_test, y_test)}
    MLP_df_gaze = pd.concat([MLP_df_gaze, pd.DataFrame(data=row_, index=np.arange(1))], ignore_index=True)


In [41]:
MLP_df_gaze[MLP_df_gaze['Test']>0.6]

,Subject,Train,Test
4,6,1.000000,0.735294
8,16,1.000000,0.617647
10,21,1.000000,0.647059
12,23,1.000000,0.617647
14,27,1.000000,0.617647
15,28,1.000000,0.647059
25,48,1.000000,0.617647
29,54,0.987179,0.647059


# MLP

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
valid_subject = extract_correct_csv.extract_only_valid_subject()
valid_subject.remove(50)
valid_subject.remove(51)

In [ ]:
# MLP mono signals
columns_mono = ['Subject', 'Feature', 'Train', 'Test']
MLP_df_mono = pd.DataFrame(columns=columns_mono)
types_ = ['hr', 'eda', 'pupil']
for sub in valid_subject:
    string_sub = extract_correct_csv.read_correct_subject_csv(sub)
    df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
    y = np.array(list([int (d > 2) for d in df_['rating']]))
    y = y[48:]

    for type_ in types_:
        X = pd.read_csv('data/features_4_2/'+type_+'/'+str(sub)+'.csv')
        X = pd.DataFrame(scaler.fit_transform(X))
        X = X[48:]
        X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123, stratify=y)
        clf = MLPClassifier(hidden_layer_sizes=(30, 20), max_iter=3000, learning_rate='adaptive', random_state=123).fit(X_train, y_train)
        row_ = {'Subject': sub,
                'Feature': type_,
                'Train': clf.score(X_train, y_train),
                'Test':clf.score(X_test, y_test)}
        MLP_df_mono = pd.concat([MLP_df_mono, pd.DataFrame(data=row_, index=np.arange(1))], ignore_index=True)


In [ ]:
mean_mono = MLP_df_mono.groupby(by='Feature', as_index=False).mean()
std_mono = MLP_df_mono.groupby(by='Feature', as_index=False).std()

In [ ]:
from tqdm import tqdm

# test tuning MLP
first_layer_neurons = np.arange(10, 150, 10)
second_layer_neurons = np.arange(10, 150, 10)
third_layer_neurons = np.arange(10, 150, 10)


best_hyperP = pd.DataFrame(columns=['first layer', 'second layer', 'third layer', 'train', 'test'])

for first in first_layer_neurons:
    for second in second_layer_neurons:
        for third in third_layer_neurons:
            mean_test = []
            mean_train = []
            for sub in valid_subject:
                string_sub = extract_correct_csv.read_correct_subject_csv(sub)
                df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
                y = np.array(list([int (d > 2) for d in df_['rating']]))
                y = y[48:]
                X1 = pd.read_csv('data/features_4_2/hr/'+str(sub)+'.csv')
                X1 = pd.DataFrame(scaler.fit_transform(X1))
                X2 = pd.read_csv('data/features_4_2/eda/'+str(sub)+'.csv')
                X2 = pd.DataFrame(scaler.fit_transform(X2))
                X3 = pd.read_csv('data/features_4_2/pupil/'+str(sub)+'.csv')
                X3 = pd.DataFrame(scaler.fit_transform(X3))
                X = pd.concat([X1, X2, X3], axis=1)
                X = X[48:]
                X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=123, stratify=y)
                clf = MLPClassifier(hidden_layer_sizes=(first, second, third),
                                    max_iter=3000,
                                    learning_rate='adaptive',
                                    random_state=123)\
                    .fit(X_train, y_train)
                mean_train.append(clf.score(X_train, y_train))
                mean_test.append(clf.score(X_test, y_test))

            row_dict = {'first layer': first,
                        'second layer': second,
                        'third layer': third,
                        'train': np.array(mean_train).mean(),
                        'test': np.array(mean_test).mean()}
            best_hyperP = pd.concat([best_hyperP, pd.DataFrame(data=row_dict, index=np.arange(1))], ignore_index=True)
best_hyperP.to_csv('output_mlp.csv')

In [ ]:
best_hyperP.loc[best_hyperP.test.idxmax()]

In [ ]:

columns = ['Subject', 'Train', 'Test']

mean_MLP_df = pd.DataFrame(columns=columns)
for sub in valid_subject:
    string_sub = extract_correct_csv.read_correct_subject_csv(sub)
    df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
    y = np.array(list([int (d > 2) for d in df_['rating']]))
    y = y[48:]


    X1 = pd.read_csv('data/features_4_2/hr/'+str(sub)+'.csv')
    X1 = pd.DataFrame(scaler.fit_transform(X1))
    X2 = pd.read_csv('data/features_4_2/eda/'+str(sub)+'.csv')
    X2 = pd.DataFrame(scaler.fit_transform(X2))
    X3 = pd.read_csv('data/features_4_2/pupil/'+str(sub)+'.csv')
    X3 = pd.DataFrame(scaler.fit_transform(X3))

    X = pd.concat([X1, X2, X3], axis=1)
    X = X[48:]
    X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=123, stratify=y)
    clf = MLPClassifier(hidden_layer_sizes=(100, 50, 20), max_iter=1500, learning_rate='adaptive', random_state=123).fit(X_train, y_train)
    row_ = {'Subject': sub,
            'Train': clf.score(X_train, y_train),
            'Test':clf.score(X_test, y_test)}
    mean_MLP_df = pd.concat([mean_MLP_df, pd.DataFrame(data=row_, index=np.arange(1))], ignore_index=True)
mean_MLP_df

In [ ]:
mean_MLP_df.Test.mean()

## Logistic regression with morph level

In [2]:
# extract morph level functions
def extract_cs(df):
    cs1, cs2 = int(df[df.shock == True].picName.unique()[0][5]), int(df[df.shock == True].picName.unique()[1][5])
    return cs1, cs2


morph_pos = 11
morph_value = 15


def extract_threat_level(df):
    MORPH_POS = 11
    MORPH_VALUE = 15
    threat_person = []
    cs1_pic, cs2_pic = extract_cs(df)

    for i in df.iterrows():
        nome = i[1].picName
        try:
            if nome[5] == str(cs1_pic) or nome[5] == str(cs2_pic):
                threat_person.append(6)
                continue
            elif int(nome[5]):
                threat_person.append(1)
                continue
        except:
            if (int(nome[MORPH_POS]) == cs1_pic) or (int(nome[MORPH_POS]) == cs2_pic):
                if nome[MORPH_VALUE] == '2':
                    threat_person.append(5)

                elif nome[MORPH_VALUE] == '4':
                    threat_person.append(4)

                elif nome[MORPH_VALUE] == '6':
                    threat_person.append(3)

                elif nome[MORPH_VALUE] == '8':
                    threat_person.append(2)
                    continue
            else:
                if nome[MORPH_VALUE] == '2':
                    threat_person.append(2)

                elif nome[MORPH_VALUE] == '4':
                    threat_person.append(3)

                elif nome[MORPH_VALUE] == '6':
                    threat_person.append(4)

                elif nome[MORPH_VALUE] == '8':
                    threat_person.append(5)
    return threat_person

In [3]:
subjects = extract_correct_csv.extract_only_valid_subject()


In [20]:
columns = ['subject', 'feature', 'fold', 'train accuracy', 'test accuracy']
results = pd.DataFrame(columns = columns)

for x in subjects:
    string_sub = extract_correct_csv.read_correct_subject_csv(x)
    df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
    df_ = df_[48:]
    y = np.array(list([int (d > 2) for d in df_['rating']]))


    X1 = pd.read_csv('data/features_4_2/hr/'+str(x)+'.csv')
    X1 = pd.DataFrame(scaler.fit_transform(X1))
    X1 = X1[48:]
    print(X1.shape)
    X2 = pd.read_csv('data/features_4_2/eda/'+str(x)+'.csv')
    X2 = pd.DataFrame(scaler.fit_transform(X2))
    X2 = X2[48:]
    print(X2.shape)
    X3 = pd.read_csv('data/features_4_2/pupil/'+str(x)+'.csv')
    X3 = pd.DataFrame(scaler.fit_transform(X3))
    X3 = X3[48:]
    print(X3.shape)
    img = np.array(extract_threat_level(df_))
    morph_level = np.zeros((img.size, img.max()))
    morph_level[np.arange(img.size), img-1] = 1
    X4 = morph_level.astype(int)
    X4 = pd.DataFrame(X4)
    print(X4)
    X = pd.concat([X1, X2, X3, X4], axis=1)

    #print(X.shape)
    break
    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=123)

    for i, (train_index, test_index) in enumerate(sss.split(X, y)):
        y = pd.DataFrame(y)
        y = y.reset_index().drop(columns=('index'))

        N_train = len(train_index)
        X_train = X.iloc[train_index, :]
        y_train =y.iloc[train_index, :]

        X_test = X.iloc[test_index, :]
        y_test =y.iloc[test_index, :]

        clf = LogisticRegression(random_state=123,max_iter=10000)
        clf.fit(X_train, np.squeeze(np.array(y_train)))

        train_ = clf.score(X_train, y_train)
        test_ = clf.score(X_test, y_test)
        dict_ = {'subject': x, 'feature': 'morph level' ,'fold':int(i), 'train accuracy': train_, 'test accuracy': test_}
        results = pd.concat([results, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)

(112, 24)
(112, 24)
(112, 28)
(112, 6)
(160, 82)


In [5]:
results.groupby(by='subject').mean(numeric_only=True)

,train accuracy,test accuracy
subject,,
2,0.794007,0.811594
4,0.670412,0.666667
10,0.696629,0.782609
41,0.771536,0.623188
43,0.868914,0.855072
44,0.707865,0.579710
45,0.921348,0.913043
46,0.655431,0.637681
47,0.910112,0.840580


## Mean over all subjects

In [6]:
results.drop(columns=['subject', 'fold']).mean(numeric_only=True)

train accuracy    0.797312
test accuracy     0.774936
dtype: float64

In [7]:
results.drop(columns=['subject', 'fold']).var(numeric_only=True)

train accuracy    0.008994
test accuracy     0.019713
dtype: float64

## Logistic regression with all signals

In [ ]:
columns = ['subject', 'feature', 'fold', 'train accuracy', 'test accuracy']
results = pd.DataFrame(columns = columns)

for x in subjects:
    string_sub = extract_correct_csv.read_correct_subject_csv(x)
    df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
    df_ = df_[48:]
    y = np.array(list([int (d > 2) for d in df_['rating']]))


    img = np.array(extract_threat_level(df_))
    morph_level = np.zeros((img.size, img.max()))
    morph_level[np.arange(img.size), img-1] = 1
    X = morph_level.astype(int)
    X = pd.DataFrame(X)
    X = X.reset_index().drop(columns='index')

    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=123)

    for i, (train_index, test_index) in enumerate(sss.split(X, y)):
        y = pd.DataFrame(y)
        y = y.reset_index().drop(columns=('index'))

        N_train = len(train_index)
        X_train = X.iloc[train_index, :]
        y_train =y.iloc[train_index, :]

        X_test = X.iloc[test_index, :]
        y_test =y.iloc[test_index, :]

        clf = LogisticRegression(random_state=123,max_iter=10000)
        clf.fit(X_train, np.squeeze(np.array(y_train)))

        train_ = clf.score(X_train, y_train)
        test_ = clf.score(X_test, y_test)
        dict_ = {'subject': x, 'feature': 'morph level' ,'fold':int(i), 'train accuracy': train_, 'test accuracy': test_}
        results = pd.concat([results, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)

## Results with our model

In [8]:
mono_im = pd.read_csv('output/FA/FA_only_image.csv')
mono_im

,subject,k,fold,train,test
0,2,2,0,0.505618,0.478261
1,2,2,1,0.617978,0.739130
2,2,2,2,0.494382,0.521739
3,2,3,0,0.213483,0.391304
4,2,3,1,0.786517,0.869565
...,...,...,...,...,...
235,55,5,1,0.674157,0.652174
236,55,5,2,0.573034,0.521739
237,55,6,0,0.292135,0.304348
238,55,6,1,0.550562,0.521739


In [ ]:
complete_image = pd.read_csv('output/FA/FA_kcv_norm_image_new.csv')